In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *
from copy import copy
ROOT.gErrorIgnoreLevel = ROOT.kError

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
def checkIfExists(files, title):
    objectAvailable = False
    hist = None
    if type(files) is list:
        file = files[0]
    else:
        file = files
    try:
        hist = file.PIDQC.Get(title)
        if not hist : hist = file.TracksQC.Get(title)
        if not hist : hist = file.ClusterQC.Get(title)
        if not hist : hist = file.PID.Get(title)
        if not hist : hist = file.Tracks.Get(title)
    except:
        objectAvailable = False
    else:
        if hist:
            objectAvailable = True
        else:
            objectAvailable = False
    if not objectAvailable:
        print(title,"not found")
    return objectAvailable
    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60]):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                           pads=True,drawOption="COLZ",maxColumns=3)
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        histo.FitSlicesY(myFunc)
        fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        fits.append(fit)
        #legend.AddEntry(fit,runList[i])
        #fit.Draw("SAME L")
    #legend.Draw()
    return fits, legend, canvas
    
def quantileProfile(hist,quantileOrder=0.5, axis="x"):
    from copy import copy
    quants = []
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
    return quants

In [ ]:
# Notebook variables
periodName = "myPeriod"
passName = "myPass"
runNumber = 123456
path = "myPath"
# Read QC root file
rootDataFile=[]
rootDataFile.append(ROOT.TFile.Open(path+"/"+periodName+"/"+passName+"/"+str(runNumber)+"_QC.root","READ"))

# TPC Async QC Report - myPeriod - myPass - Run 123456
### RCT Tables
+ [2022](https://docs.google.com/spreadsheets/d/14vXFYVx3oVE8wgJKmIBhjvAt6NpGp7D6H4AmBM9E0Cw/edit#gid=0), [2023](https://docs.google.com/spreadsheets/d/1YBQLXWwwc3aC3B_PYVpFkTgEP0n6u1ovtYfiCOMWnTc/edit?pli=1#gid=0), [2023 PbPb](https://docs.google.com/spreadsheets/d/1vsl-pkLdUoNXZm6muPyMENWdDSKM7UDtQLcXc2B9QhE/edit#gid=492527515)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Per-Pad-(Clusters)" data-toc-modified-id="Per-Pad-(Clusters)-1">Per Pad (Clusters)</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#nClusters" data-toc-modified-id="nClusters-1.0.1">nClusters</a></span></li><li><span><a href="#Q_Max" data-toc-modified-id="Q_Max-1.0.2">Q_Max</a></span></li><li><span><a href="#Q_Tot" data-toc-modified-id="Q_Tot-1.0.3">Q_Tot</a></span></li><li><span><a href="#Time_Bin" data-toc-modified-id="Time_Bin-1.0.4">Time_Bin</a></span></li><li><span><a href="#Sigma_Pad" data-toc-modified-id="Sigma_Pad-1.0.5">Sigma_Pad</a></span></li><li><span><a href="#Sigma_Time" data-toc-modified-id="Sigma_Time-1.0.6">Sigma_Time</a></span></li></ul></li></ul></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-2">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions" data-toc-modified-id="Geometrical-distributions-2.1">Geometrical distributions</a></span><ul class="toc-item"><li><span><a href="#Phi" data-toc-modified-id="Phi-2.1.1">Phi</a></span></li><li><span><a href="#Eta" data-toc-modified-id="Eta-2.1.2">Eta</a></span></li><li><span><a href="#Eta-vs-phi" data-toc-modified-id="Eta-vs-phi-2.1.3">Eta vs phi</a></span></li></ul></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-2.2">Track properties</a></span><ul class="toc-item"><li><span><a href="#nClusters" data-toc-modified-id="nClusters-2.2.1">nClusters</a></span></li><li><span><a href="#nClusters-vs-eta" data-toc-modified-id="nClusters-vs-eta-2.2.2">nClusters vs eta</a></span></li><li><span><a href="#nClusters-vs-pT" data-toc-modified-id="nClusters-vs-pT-2.2.3">nClusters vs pT</a></span></li><li><span><a href="#pT" data-toc-modified-id="pT-2.2.4">pT</a></span></li><li><span><a href="#Q/pT" data-toc-modified-id="Q/pT-2.2.5">Q/pT</a></span></li><li><span><a href="#Sign" data-toc-modified-id="Sign-2.2.6">Sign</a></span></li></ul></li><li><span><a href="#Energy-Loss" data-toc-modified-id="Energy-Loss-2.3">Energy Loss</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-for-MIPs" data-toc-modified-id="dEdxTot-for-MIPs-2.3.1">dEdxTot for MIPs</a></span></li><li><span><a href="#dEdxTot-vs-p" data-toc-modified-id="dEdxTot-vs-p-2.3.2">dEdxTot vs p</a></span></li><li><span><a href="#dEdxTotMip-vs-sector-(phi)" data-toc-modified-id="dEdxTotMip-vs-sector-(phi)-2.3.3">dEdxTotMip vs sector (phi)</a></span></li><li><span><a href="#dEdxTotMip-vs-nClusters" data-toc-modified-id="dEdxTotMip-vs-nClusters-2.3.4">dEdxTotMip vs nClusters</a></span></li><li><span><a href="#dEdxTotMip-vs-sin(phi)" data-toc-modified-id="dEdxTotMip-vs-sin(phi)-2.3.5">dEdxTotMip vs sin(phi)</a></span></li><li><span><a href="#dEdxTotMip-vs-tan(lambda)" data-toc-modified-id="dEdxTotMip-vs-tan(lambda)-2.3.6">dEdxTotMip vs tan(lambda)</a></span></li><li><span><a href="#dEdxMax-for-MIPs" data-toc-modified-id="dEdxMax-for-MIPs-2.3.7">dEdxMax for MIPs</a></span></li><li><span><a href="#dEdxMax-vs-p" data-toc-modified-id="dEdxMax-vs-p-2.3.8">dEdxMax vs p</a></span></li><li><span><a href="#dEdxMaxMip-vs-sector-(phi)" data-toc-modified-id="dEdxMaxMip-vs-sector-(phi)-2.3.9">dEdxMaxMip vs sector (phi)</a></span></li><li><span><a href="#dEdxMaxMip-vs-nClusters" data-toc-modified-id="dEdxMaxMip-vs-nClusters-2.3.10">dEdxMaxMip vs nClusters</a></span></li><li><span><a href="#dEdxMaxMip-vs-sin(phi)" data-toc-modified-id="dEdxMaxMip-vs-sin(phi)-2.3.11">dEdxMaxMip vs sin(phi)</a></span></li><li><span><a href="#dEdxMaxMip-vs-tan(lambda)" data-toc-modified-id="dEdxMaxMip-vs-tan(lambda)-2.3.12">dEdxMaxMip vs tan(lambda)</a></span></li></ul></li></ul></li></ul></div>

## Per Pad (Clusters)

#### nClusters

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

#### Q_Max

In [ ]:
%jsroot on
objectName = "c_Q_Max"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

#### Q_Tot

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

#### Time_Bin

In [ ]:
%jsroot on
objectName = "c_Time_Bin"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

#### Sigma_Pad

In [ ]:
%jsroot on
objectName = "c_Sigma_Pad"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

#### Sigma_Time

In [ ]:
%jsroot on
objectName = "c_Sigma_Time"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    c.Draw()

## Tracks

### Geometrical distributions

#### Phi

In [ ]:
%jsroot on
objectName="hPhiAside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    canvas.SetCanvasSize(1000,400)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    canvas.SetCanvasSize(1000,400)
    canvas.Draw()

#### Eta

In [ ]:
%jsroot on
objectName="hEta;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,xAxisRange = [-1.1,1.1])
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### Eta vs phi

In [ ]:
%jsroot on
objectName="h2DEtaPhi;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-1,1])
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-1,(j*2*math.pi)/18,1)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,1.05,str(j))
    canvas.SetCanvasSize(1000,400)
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    #hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

### Track properties

#### nClusters

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,xAxisRange = [50,160])
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### nClusters vs eta

In [ ]:
%jsroot on
objectName="h2DNClustersEta;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ")#,xAxisRange = [-1,1],yAxisRange = [60,180])
    canvas.SetCanvasSize(700,400)
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME")
    hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### nClusters vs pT

In [ ]:
%jsroot on
objectName="h2DNClustersPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logx",yAxisRange = [60,180])
    canvas.SetCanvasSize(700,400)
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME")
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### nClusters vs phi (A-Side)

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ")#,yAxisRange = [60,180])
    pad1.cd()
    canvas.SetCanvasSize(1000,400)
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,60,(j*2*math.pi)/18,180)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,175,str(j))
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME")
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### nClusters vs phi (C-Side)

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ")#,yAxisRange = [60,180])
    pad1.cd()
    canvas.SetCanvasSize(1000,400)
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,60,(j*2*math.pi)/18,180)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,175,str(j))
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME")
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### pT

In [ ]:
%jsroot on
objectName="hPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,log="logxy")
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### Q/pT

In [ ]:
%jsroot on
objectName="hQOverPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### Sign

In [ ]:
%jsroot on
objectName="hSign;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="HIST")
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

### DCAr vs phi

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

### Energy Loss

#### dEdxTot for MIPs

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
if checkIfExists(rootDataFile,objectName):
    quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 48<histogram.GetFunction(\"gaus\").GetParameter(1)<52",printQuality=True)
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,check=quality)
    
    myFit = hist[0].GetFunction("gaus");
    
    pad1.cd()
    #Draw target 50
    target = ROOT.TLine(50,hist[0].GetMinimum(),50,hist[0].GetMaximum())
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.cd()
    # Draw fit stats
    fitMean = ROOT.TText(.55,.8,"Fit mean = "+f'{myFit.GetParameter(1):.3f}')
    fitMean.SetTextAlign(22)
    fitMean.SetTextFont(43)
    fitMean.Draw()
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### dEdxTotMip vs sector (phi)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(1000,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()

#### dEdxTot vs p

In [ ]:
%jsroot on
objectName="hdEdxTotVsP_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz")
    pad1.cd()
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### dEdxTotMip vs nClusters

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange = [55,180],yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(55,50,153,50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(700,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()#### dEdxTotMip vs sector (phi)

#### dEdxTotMip vs tan(lambda)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(700,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()

#### dEdxTotMip vs sin(phi)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(700,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()

#### dEdxMax for MIPs

In [ ]:
%jsroot on
objectName="hdEdxMaxMIP_TPC"
if checkIfExists(rootDataFile,objectName):
    quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 48<histogram.GetFunction(\"gaus\").GetParameter(1)<52",printQuality=True)
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,check=quality)
    
    myFit = hist[0].GetFunction("gaus");
    
    pad1.cd()
    #Draw target 50
    target = ROOT.TLine(50,hist[0].GetMinimum(),50,hist[0].GetMaximum())
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.cd()
    # Draw fit stats
    fitMean = ROOT.TText(.55,.8,"Fit mean = "+f'{myFit.GetParameter(1):.3f}')
    fitMean.SetTextAlign(22)
    fitMean.SetTextFont(43)
    fitMean.Draw()
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### dEdxMax vs p

In [ ]:
%jsroot on
objectName="hdEdxMaxVsP_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz")
    pad1.cd()
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

#### dEdxMaxMip vs sector (phi)

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsSec_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(1000,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()

#### dEdxMaxMip vs nClusters

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsNcl_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange = [55,180],yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(55,50,153,50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(1000,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()#### dEdxTotMip vs sector (phi)

#### dEdxMaxMip vs tan(lambda)

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsTgl_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(1000,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()

#### dEdxMaxMip vs sin(phi)

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsSnp_TPC;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,80])
    pad1.cd()
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(4)
    target.Draw()
    canvas.SetCanvasSize(1000,400)
    hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    canvas.Draw()